In [1]:
import math
import geopy.distance
from dis import dis
import math
import geopandas as gpd
import numpy
from shapely import wkt
from shapely import wkb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim
%matplotlib inline


train_data = pd.read_csv('data/stores_train.csv',nrows=1000)
busstops = pd.read_csv('data/busstops_norway.csv')
grunnkrets_age = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_households = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_income = pd.read_csv('data/grunnkrets_income_households.csv')
grunnkrets_stripped = pd.read_csv('data/grunnkrets_norway_stripped.csv')
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')

In [ ]:
#Julia

# Replace NaN in mall_name column with 'No mall'
train_data.mall_name = train_data.mall_name.fillna('No mall')
train_data.address = train_data.address.fillna('No address')
train_data.chain_name = train_data.chain_name.fillna('No chain')

# 'store_name', 'year', 'sales_channel_name', 'address' columns are redundant, remove them
train_data = train_data.drop('store_name',axis=1)
train_data = train_data.drop('year',axis=1)
train_data = train_data.drop('sales_channel_name',axis=1)
train_data = train_data.drop('address',axis=1)

# Make new column for less specified plaace hierarchy to group data together
train_data = pd.merge(train_data, plaace_hierarchy[['plaace_hierarchy_id', 'lv1', 'lv2', 'lv3']], on='plaace_hierarchy_id', how='outer')
train_data.isnull().sum().sum()

# Dummy variable for mall or no mall
train_data.loc[train_data['mall_name'].str.contains("No mall", na=False),'mall_dummy'] = 0
train_data.loc[~(train_data['mall_name'].str.contains("No mall", na=False)),'mall_dummy'] = 1
train_data.drop(['mall_name'],axis=1, inplace=True)

# Sum together the number of people in each grunnkrets
grunnkrets_age = grunnkrets_age.drop_duplicates(subset=['grunnkrets_id'], keep='last') # if there is value for 2016 we keep it, otherwise 2015
grunnkrets_age = grunnkrets_age.fillna(0)
grunnkrets_age = grunnkrets_age.drop('year',axis=1)
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(str)
grunnkrets_age['total_nbr_people'] = grunnkrets_age.sum(axis=1) # total number of inhabitants
grunnkrets_age['group1'] = grunnkrets_age.iloc[:,1:11].sum(axis=1) # 0-9 years old
grunnkrets_age['group2'] = grunnkrets_age.iloc[:,11:21].sum(axis=1) # 10-19 years old etc
grunnkrets_age['group3'] = grunnkrets_age.iloc[:,21:31].sum(axis=1)
grunnkrets_age['group4'] = grunnkrets_age.iloc[:,31:41].sum(axis=1)
grunnkrets_age['group5'] = grunnkrets_age.iloc[:,41:51].sum(axis=1)
grunnkrets_age['group6'] = grunnkrets_age.iloc[:,51:61].sum(axis=1)
grunnkrets_age['group7'] = grunnkrets_age.iloc[:,61:71].sum(axis=1)
grunnkrets_age['group8'] = grunnkrets_age.iloc[:,71:81].sum(axis=1)
grunnkrets_age['group9'] = grunnkrets_age.iloc[:,81:92].sum(axis=1) # 80-90 years old
grunnkrets_age['grunnkrets_id'] = grunnkrets_age['grunnkrets_id'].astype(int)
train_data = pd.merge(train_data, grunnkrets_age[['grunnkrets_id', 'total_nbr_people', 'group1', 'group2', 'group3', 'group4', 'group5', 'group6', 'group7', 'group8', 'group9']], on='grunnkrets_id', how='left')


# Number of people per store type (lv2) and grunnkrets

number_stores = train_data['grunnkrets_id'].value_counts().rename_axis('grunnkrets_id').reset_index(name='store_counts_total') # Not including NaN (stores without a grunnkrets_id)
grunnkrets_stripped = pd.merge(grunnkrets_stripped, number_stores[['grunnkrets_id', 'store_counts_total']], on='grunnkrets_id', how='left')
grunnkrets_stripped.store_counts_total = grunnkrets_stripped.store_counts_total.fillna(0)
grunnkrets_stripped = pd.merge(grunnkrets_stripped, grunnkrets_age[['grunnkrets_id', 'total_nbr_people']], on='grunnkrets_id', how='left')
grunnkrets_stripped['nbr_people_per_store_in_grunnkrets'] = grunnkrets_stripped['total_nbr_people']/grunnkrets_stripped['store_counts_total']
train_data = pd.merge(train_data, grunnkrets_stripped[['grunnkrets_id', 'nbr_people_per_store_in_grunnkrets']], on='grunnkrets_id', how='left')

# Calc how mani is in lvl2
counts = train_data[["store_id", "grunnkrets_id", "lv2"]].groupby(
    ["grunnkrets_id", "lv2"]
).count().reset_index()
counts.columns = ["grunnkrets_id", "lv2", "counts_gr_lv2"]
train_data.merge(counts, how="left", on=["grunnkrets_id", "lv2"])

#Fix missing values
train_data.update(train_data[['total_nbr_people','group1','group2','group3','group4','group5','group6','group7','group8','group9']].fillna(0))
train_data.update(train_data[['nbr_people_per_store_in_grunnkrets','nbr_people_per_km2']].fillna(0))

#Haakon


#number of stores within x radius
#sklarn har noe some heter balltree
#brukes til til å søke opp k neerest neighbors. 
def stores_within_distance(distance):
    train_data['num_stores_within_distance'] = 0
    for index in range(len(train_data)):
        lvl1 = train_data._get_value(index,'lv2')
        lat1 = train_data._get_value(index,'lat')
        lon1 = train_data._get_value(index,'lon')
        count = 0
        for inx in range(len(train_data)):
            lvl2 = train_data._get_value(inx,'lv2')
            lat2 = train_data._get_value(inx,'lat')
            lon2 = train_data._get_value(inx,'lon')
            try:
                dist = geopy.distance.geodesic((lat1, lon1), (lat2, lon2)).km #some values have nan in them. must be fixed. 
                print(lat1, lat2, lon1, lon2)
            except:
                print(lat1, lat2, lon1, lon2)

            if dist < distance and lvl1 == lvl2:
                count += 1
        train_data._set_value(index, 'num_stores_within_distance', count- 1)

#busstops
#tar alle busstoppene balltree hele pakka. 
def busstops_features():
    train_data['distance_to_closest_buss_station'] = 0
    train_data['importance_level'] = 0
    train_data['closest_local_knutepunkt'] = 0

    gdf = gpd.GeoDataFrame(busstops)
    gdf['geometry'] = gpd.GeoSeries.from_wkt(gdf['geometry'])

    for index in range(len(gdf)):
        closest = 100000 #large number
        importance = ""
        closest_high_importance = 0
        lat = train_data._get_value(index,'lat')
        lon = train_data._get_value(index,'lon')
        for numbuss in range(len(gdf)):
            val = gdf._get_value(index,'geometry')
            dist = geopy.distance.geodesic((val.x, val.y), (lat, lon)).km
            if dist < closest:
                closest = dist
                importance = gdf._get_value(index,'importance_level')
            if gdf._get_value(index,'importance_level') == 'Lokalt knutepunkt':
                closest_high_importance = dist
        train_data._set_value(index, 'distance_to_closest_buss_station', closest)
        train_data._set_value(index, 'importance_level', importance)
        train_data._set_value(index, 'closest_local_knutepunkt', closest_high_importance)

#usikker på om det er lov. 
# ikke stor forskjell på grunnkrets og 
def postcode():
    geolocator = Nominatim(user_agent="geoapiExercises")
    # Display
    train_data = pd.read_csv('data/stores_train.csv', nrows= 10)
    df = train_data
    df['postcode'] = 0
    for index in range(len(df)):
        lat = df._get_value(index,'lat')
        lon = df._get_value(index,'lon')
        location = geolocator.reverse(str(lat)+","+str(lon))
        address = location.raw['address']
        zipcode = address.get('postcode')
        df._set_value(index, 'postcode', str(zipcode))
    df['postcode']= df.postcode.astype('category')